In [22]:
import tensorflow as tf
import numpy as np

import pickle

tf.reset_default_graph()

vocab_size = 17322
beam_width = 4

time = 1
n_grams = pickle.load(open('n_grams_test.pkl', 'rb'))

n_grams_tf = tf.constant(n_grams[time])

In [23]:
import pandas as pd
df = pd.DataFrame(n_grams[time], columns=['a', 'b'])

In [24]:
#beam = tf.constant([[6178, 5729, 15173], 
#                    [2341, 16073, 5683], 
#                    [434, 9055, 2692], 
#                    [6178, 14056, 7441]]) #Current predicted steps, will specify at run time

In [46]:
beam = tf.constant([[ 617],
        [ 440032],
        [10359],
        [1167832]]) #Current predicted steps, will specify at run time

In [47]:
beam_pad = tf.pad(beam, [[0, 0], [0, 1]], mode='CONSTANT', constant_values=0)

initial_beam_step = tf.constant(0)

n_beams = tf.constant(beam_width) #will specify at run time

#When you specify the axis of concat such a rank must exist! 
#Thus cannot specify axis=1 if the rank is 0!
scatter_base = tf.constant([vocab_size]) #Size of dict for scatter base will specify at run time
concat_base = tf.constant([[1.0 for i in range(vocab_size)]], dtype=tf.float64) #concat base will specify vocab size at run time

In [48]:
def anti_lm_body(test, n_grams_tf, beam_pad, current_beam, n_beams):
    #This loops on each beam individually
      
    def grab_probs(n_grams_tf, y_equal_2): 
    
        #print ('Grab Probabilities')
        y_args = tf.where(y_equal_2)

        #Grab the n_gram sequences that are matched
        y_diff_gather = tf.gather(n_grams_tf, y_args)
        #Take the last token in each sequence and count their unique occurrences

        last_token_pos = tf.shape(y_diff_gather)[-1]

        y_last_token = y_diff_gather[:,0][:,last_token_pos-1]
        y_last_token_unique = tf.unique_with_counts(y_last_token)

        total_count = tf.reduce_sum(y_last_token_unique.count)

        indices = tf.reshape(y_last_token_unique.y, [tf.shape(y_last_token_unique.y)[0], 1])

        test_add_result = tf.scatter_nd(indices=indices, 
                    updates=y_last_token_unique.count, shape=scatter_base)/total_count

        test_add_result = tf.log(test_add_result + 10e-10)

        return tf.reshape(test_add_result, [1, 17322])

    def dump_zeros(n_grams_tf, y_equal_2): 
        #print ('No matches found')
        return tf.constant([[0. for i in range(vocab_size)]], dtype=tf.float64)

    #Find where current beam matches n_gram sequence up to current seq pos, cast as int
    y_test = tf.to_int32(tf.equal(n_grams_tf, beam_pad[current_beam]))
    #Reduce across the length of the beam 
    y_test_reduce_sum = tf.reduce_sum(y_test, axis=1)

    y_empty = tf.reduce_sum(y_test_reduce_sum)

    #Find args where the beam is matched to the n_gram combinations
    y_equal_2  = tf.equal(time, y_test_reduce_sum)

    #Why does cond proceed down both paths?
    test_add = tf.cond(tf.equal(0, tf.reduce_sum(tf.to_int32(y_equal_2))),
        true_fn=lambda: dump_zeros(n_grams_tf, y_equal_2),
        false_fn=lambda : grab_probs(n_grams_tf, y_equal_2))

    test = tf.concat([test, test_add], axis=0)
    
    return test, n_grams_tf, beam_pad, current_beam+1, n_beams

def anti_lm_condition(test, n_grams_tf, beam_pad, current_beam, n_beams):
    return current_beam < n_beams #number of beams, i.e. beam width, will specify at run time

anti_lm_outputs = tf.while_loop(anti_lm_condition, anti_lm_body, 
                        [concat_base, n_grams_tf, beam_pad, initial_beam_step, n_beams], 
                        shape_invariants=[tf.TensorShape([None, vocab_size]), 
                                          n_grams_tf.get_shape(),
                                      beam_pad.get_shape(), initial_beam_step.get_shape(), n_beams.get_shape()])

anti_beam_probs = anti_lm_outputs[0][1:]

In [49]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    #q = sess.run(outputs[0].stack())
    q = sess.run(anti_lm_outputs)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [44]:
np.max(q[0]), np.max(q[1]), np.max(q[2]), np.max(q[3])

(1.0, 17322, 617123, 4)

In [50]:
np.argsort(q[0])

array([[    0, 11538, 11539, ...,  5783,  5769, 17321],
       [    0, 11537, 11538, ...,  5773, 17321, 15979],
       [    0, 11538, 11539, ...,  5783,  5769, 17321],
       [    0, 11524, 11525, ..., 16911, 10036,  6178],
       [    0, 11538, 11539, ...,  5783,  5769, 17321]], dtype=int64)